# Faster R-CNN experiments
This file contains experiments for my Faster R-CNN implementation. To access GPUs/TPUs, I ran this file in Google Colaboratory.

To pull from the GitHub repository:
```
%cd drive/MyDrive/Conservation\ Research/Code/counting-cranes
!git pull
```
----

In [1]:
#Will have to restart runtime after running this cell!
!pip install -r "/content/drive/MyDrive/Conservation Research/Code/counting-cranes/requirements.txt"

     |████████████████████████████████| 2.2MB 9.5MB/s 
     |████████████████████████████████| 17.4MB 170kB/s 
     |████████████████████████████████| 102kB 13.2MB/s 
     |████████████████████████████████| 15.3MB 198kB/s 
     |████████████████████████████████| 11.6MB 49.3MB/s 
     |████████████████████████████████| 808kB 44.1MB/s 
     |████████████████████████████████| 38.2MB 67kB/s 
     |████████████████████████████████| 112kB 47.6MB/s 
     |████████████████████████████████| 276kB 54.1MB/s 
     |████████████████████████████████| 829kB 38.5MB/s 
     |████████████████████████████████| 10.6MB 40.3MB/s 
     |████████████████████████████████| 61kB 10.1MB/s 
     |████████████████████████████████| 1.3MB 40.3MB/s 
     |████████████████████████████████| 296kB 52.5MB/s 
     |████████████████████████████████| 143kB 41.7MB/s 
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491070 sha256=7315e0b3520477b651029503a9ea640fce7549f09dc0e3cb8b15eb5c12867246
  Stored

In [1]:
import json

config = json.load(open('/content/drive/MyDrive/Conservation Research/Code/counting-cranes/config.json', 'r'))
DATA_FP = config['data_filepath_colab']
CODE_FP = config['code_filepath_colab']
SEED = config['random_seed']
HYPERPARAMETERS = config['faster_rcnn_params']

In [2]:
import sys
import os
sys.path.append(CODE_FP) 
sys.path.append(os.path.join(CODE_FP, 'object_detection'))

from faster_rcnn_model import *
from bird_dataset import *

from torch.utils.data import DataLoader
from pytorch_lightning import seed_everything, Trainer

## Trying to get model training working

In [3]:
#Setting our random seed for all operaitons (PyTorch, numpy, python.random)
seed_everything(SEED);

Global seed set to 1693


In [4]:
#Creating a dataset and putting it into a dataloader
bird_dataset = BirdDataset(root_dir = DATA_FP, transforms = get_transforms(train = True)) #no train/test split yet...
bird_dataloader = DataLoader(bird_dataset, batch_size = HYPERPARAMETERS['batch_size'], shuffle = True, collate_fn = collate_w_tiles) 

In [5]:
#Instantiating the model
model = get_faster_rcnn(num_classes = 2) #class for birds + class for background?
pl_model = FasterRCNNLightning(model = model, lr = HYPERPARAMETERS['learning_rate'])
pl_model #checking out the model summary

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


FasterRCNNLightning(
  (model): FasterRCNN(
    (transform): GeneralizedRCNNTransform(
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        Resize(min_size=(800,), max_size=1333, mode='bilinear')
    )
    (backbone): BackboneWithFPN(
      (body): IntermediateLayerGetter(
        (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (bn1): FrozenBatchNorm2d(64, eps=0.0)
        (relu): ReLU(inplace=True)
        (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (layer1): Sequential(
          (0): Bottleneck(
            (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn1): FrozenBatchNorm2d(64, eps=0.0)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): FrozenBatchNorm2d(64, eps=0.0)
            (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      

In [6]:
#Training the model, following the procedure in PyTorch Lightning's documentation
trainer = Trainer(gpus = 1, max_epochs = 2)
trainer.fit(pl_model, train_dataloader = bird_dataloader) #STOPPED HERE!

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: you defined a validation_step but have no val_dataloader. Skipping val loop
  warnings.warn(*args, **kwargs)
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-------------------------------------
0 | model | FasterRCNN | 41.3 M
-------------------------------------
41.1 M    Trainable params
222 K     Non-trainable params
41.3 M    Total params
165.197   Total estimated model params size (MB)


Global seed set to 1693


ValueError: ignored

In [23]:
#THE ALBUMENTATION CROP IS GENERATING INVALID BBOXES!
num_degen = 0
for i, data in enumerate(bird_dataloader):
  print('On image', i)
  images, targets = data
  for d in targets:
    for b in d['boxes'].tolist():
      xmin, ymin, xmax, ymax = b
      if xmin >= xmax or ymin >= ymax:
        num_degen += 1
print(f'We have {num_degen} invalid bboxes')

On image 0
On image 1
On image 2
On image 3
On image 4
On image 5
On image 6
On image 7
On image 8
On image 9
On image 10
On image 11
On image 12
On image 13
On image 14
On image 15
On image 16
On image 17
On image 18
On image 19
On image 20
On image 21
On image 22
On image 23
On image 24
On image 25
On image 26
On image 27
On image 28
On image 29
On image 30
On image 31
On image 32
On image 33
We have 14 invalid bboxes
